In [1]:
# Install AgentChat and OpenAI client from Extensions
%pip install "autogen-agentchat" "autogen-ext[openai,azure,web-surfer]" crawl4ai
!playwright install

  Using cached autogen_agentchat-0.4.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached Crawl4AI-0.4.248-py3-none-any.whl.metadata (29 kB)
  Using cached autogen_ext-0.4.5-py3-none-any.whl.metadata (5.5 kB)
  Using cached autogen_core-0.4.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached opentelemetry_api-1.30.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached pillow-11.1.0-cp313-cp313-macosx_10_13_x86_64.whl.metadata (9.1 kB)
  Using cached protobuf-5.29.3-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached azure_ai_inference-1.0.0b8-py3-none-any.whl.metadata (34 kB)
  Using cached azure_core-1.32.0-py3-none-any.whl.metadata (39 kB)
  Using cached azure_identity-1.19.0-py3-none-any.whl.metadata (80 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (

In [2]:
# Load .env file
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

agent = AssistantAgent("assistant", AzureOpenAIChatCompletionClient(
    model="gpt-4o", 
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["OPENAI_API_VERSION"],
))
result = await agent.run(task="Say 'Hello World!'")
print(result)

ModuleNotFoundError: No module named 'autogen_agentchat'

In [3]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

model_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o", 
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["OPENAI_API_VERSION"],
)
assistant = AssistantAgent("assistant", model_client)
web_surfer = MultimodalWebSurfer("web_surfer", model_client)
user_proxy = UserProxyAgent("user_proxy")
termination = TextMentionTermination("exit") # Type 'exit' to end the conversation.
team = RoundRobinGroupChat([web_surfer, assistant, user_proxy], termination_condition=termination)
await Console(team.run_stream(task="Find information about AutoGen and write a short summary."))

---------- user ----------
Find information about AutoGen and write a short summary.
---------- web_surfer ----------
I typed 'AutoGen information' into '0 個字元，共 2000 個字元'.

 Here is a screenshot of the webpage: [AutoGen information - 搜尋](https://www.bing.com/search?q=AutoGen+information&form=QBLH&sp=-1&lq=0&pq=&sc=0-0&qs=n&sk=&cvid=37948A388ED54E0DAA11DFB045FBEF55&ghsh=0&ghacc=0&ghpl=).
 The viewport shows 36% of the webpage, and is positioned at the top of the page 
The following metadata was extracted from the webpage:

{
    "meta_tags": {
        "referrer": "origin-when-cross-origin",
        "SystemEntropyOriginTrialToken": "A5is4nwJJVnhaJpUr1URgj4vvAXSiHoK0VBbM9fawMskbDUj9WUREpa3JzGAo6xd1Cp2voQEG1h6NQ71AsMznU8AAABxeyJvcmlnaW4iOiJodHRwczovL3d3dy5iaW5nLmNvbTo0NDMiLCJmZWF0dXJlIjoiTXNVc2VyQWdlbnRMYXVuY2hOYXZUeXBlIiwiZXhwaXJ5IjoxNzUzNzQ3MjAwLCJpc1N1YmRvbWFpbiI6dHJ1ZX0=",
        "og:description": "Bing \u7684\u667a\u6167\u578b\u641c\u5c0b\u53ef\u8b93\u60a8\u66f4\u8f15\u800c\u6613\u8

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Find information about AutoGen and write a short summary.', type='TextMessage'), MultiModalMessage(source='web_surfer', models_usage=RequestUsage(prompt_tokens=2469, completion_tokens=48), content=['I typed \'AutoGen information\' into \'0 個字元，共 2000 個字元\'.\n\n Here is a screenshot of the webpage: [AutoGen information - 搜尋](https://www.bing.com/search?q=AutoGen+information&form=QBLH&sp=-1&lq=0&pq=&sc=0-0&qs=n&sk=&cvid=37948A388ED54E0DAA11DFB045FBEF55&ghsh=0&ghacc=0&ghpl=).\n The viewport shows 36% of the webpage, and is positioned at the top of the page \nThe following metadata was extracted from the webpage:\n\n{\n    "meta_tags": {\n        "referrer": "origin-when-cross-origin",\n        "SystemEntropyOriginTrialToken": "A5is4nwJJVnhaJpUr1URgj4vvAXSiHoK0VBbM9fawMskbDUj9WUREpa3JzGAo6xd1Cp2voQEG1h6NQ71AsMznU8AAABxeyJvcmlnaW4iOiJodHRwczovL3d3dy5iaW5nLmNvbTo0NDMiLCJmZWF0dXJlIjoiTXNVc2VyQWdlbnRMYXVuY2hOYXZUeXBlIi

In [6]:
await Console(team.run_stream(task="Summarize all news on HK01 in yesterday in cantonese"))

---------- user ----------
Summarize all news on HK01 in yesterday in cantonese
---------- web_surfer ----------
The webpage "最新即時新聞" from 香港01 provides a comprehensive overview of various current news categories, including local news, international affairs, lifestyle, and economics. A highlighted feature advises readers on coping with flight anxiety, suggesting three methods taught by doctors. Additionally, the site reports on a lunar surface observation expected to reveal an "X" shape, with expert recommendations on the best viewing times. Further coverage includes news about new drugs expected to enhance profitability for Hutchison's pharmaceutical division, and educational insights on helping children readjust to school routines. The page serves as a hub for the latest news and updates across different domains relevant to Hong Kong and international audiences.
---------- assistant ----------
以下是香港01昨天的主要新聞摘要：

1. **健康旅遊**: 提供旅遊指南，針對搭飛機時感到緊張的旅客，分享醫生建議的三招應對方法，以克服飛行恐懼。

2. **天文觀察**: 介

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Summarize all news on HK01 in yesterday in cantonese', type='TextMessage'), TextMessage(source='web_surfer', models_usage=RequestUsage(prompt_tokens=21334, completion_tokens=179), content='The webpage "最新即時新聞" from 香港01 provides a comprehensive overview of various current news categories, including local news, international affairs, lifestyle, and economics. A highlighted feature advises readers on coping with flight anxiety, suggesting three methods taught by doctors. Additionally, the site reports on a lunar surface observation expected to reveal an "X" shape, with expert recommendations on the best viewing times. Further coverage includes news about new drugs expected to enhance profitability for Hutchison\'s pharmaceutical division, and educational insights on helping children readjust to school routines. The page serves as a hub for the latest news and updates across different domains relevant to Hong Kong

In [7]:
await Console(team.run_stream(task="Summarize all news about GenAI on TechCrunch in yesterday in cantonese"))

---------- user ----------
Summarize all news about GenAI on TechCrunch in yesterday in cantonese
---------- web_surfer ----------
I typed 'GenAI news site:techcrunch.com' into the browser search bar.

 Here is a screenshot of the webpage: [GenAI news site:techcrunch.com - 搜尋](https://www.bing.com/search?q=GenAI+news+site%3Atechcrunch.com&FORM=QBLH&rdr=1&rdrig=6C859721543342B28550EBA4C5E6C379).
 The viewport shows 35% of the webpage, and is positioned at the top of the page 
The following metadata was extracted from the webpage:

{
    "meta_tags": {
        "referrer": "origin-when-cross-origin",
        "SystemEntropyOriginTrialToken": "A5is4nwJJVnhaJpUr1URgj4vvAXSiHoK0VBbM9fawMskbDUj9WUREpa3JzGAo6xd1Cp2voQEG1h6NQ71AsMznU8AAABxeyJvcmlnaW4iOiJodHRwczovL3d3dy5iaW5nLmNvbTo0NDMiLCJmZWF0dXJlIjoiTXNVc2VyQWdlbnRMYXVuY2hOYXZUeXBlIiwiZXhwaXJ5IjoxNzUzNzQ3MjAwLCJpc1N1YmRvbWFpbiI6dHJ1ZX0=",
        "og:description": "Bing \u7684\u667a\u6167\u578b\u641c\u5c0b\u53ef\u8b93\u60a8\u66f4\u8f15\u800c\u

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Summarize all news about GenAI on TechCrunch in yesterday in cantonese', type='TextMessage'), MultiModalMessage(source='web_surfer', models_usage=RequestUsage(prompt_tokens=10049, completion_tokens=46), content=['I typed \'GenAI news site:techcrunch.com\' into the browser search bar.\n\n Here is a screenshot of the webpage: [GenAI news site:techcrunch.com - 搜尋](https://www.bing.com/search?q=GenAI+news+site%3Atechcrunch.com&FORM=QBLH&rdr=1&rdrig=6C859721543342B28550EBA4C5E6C379).\n The viewport shows 35% of the webpage, and is positioned at the top of the page \nThe following metadata was extracted from the webpage:\n\n{\n    "meta_tags": {\n        "referrer": "origin-when-cross-origin",\n        "SystemEntropyOriginTrialToken": "A5is4nwJJVnhaJpUr1URgj4vvAXSiHoK0VBbM9fawMskbDUj9WUREpa3JzGAo6xd1Cp2voQEG1h6NQ71AsMznU8AAABxeyJvcmlnaW4iOiJodHRwczovL3d3dy5iaW5nLmNvbTo0NDMiLCJmZWF0dXJlIjoiTXNVc2VyQWdlbnRMYXVuY2hOYXZU

In [15]:
from crawl4ai import AsyncWebCrawler
from autogen_core.tools import FunctionTool
from autogen_core.tool_agent import ToolAgent

async def get_webpage_text(webpage_url: str):
    async with AsyncWebCrawler(verbose=True,headless=True) as crawler:
        result = await crawler.arun(
            url=webpage_url,
            bypass_cache=False,
            verbose=False,
        )
        return result.markdown

fetch_website = FunctionTool(get_webpage_text, description="Fetch website content to markdown from url.")

model_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o", 
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["OPENAI_API_VERSION"],
)
assistant = AssistantAgent("assistant", model_client)
web_surfer = MultimodalWebSurfer("web_surfer", model_client)
user_proxy = UserProxyAgent("user_proxy")
fetch_website_agent = ToolAgent("Fetch a website content to be markdown from url", tool=fetch_website)
termination = TextMentionTermination("exit") # Type 'exit' to end the conversation.
team = RoundRobinGroupChat([web_surfer, fetch_website_agent, assistant, user_proxy], termination_condition=termination)

await Console(team.run_stream(task="Summarize all news about GenAI on TechCrunch in yesterday in cantonese"))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kan/my-workspaces/genai-samples/.venv/lib/python3.12/site-packages/autogen_ext/agents/web_surfer/page_script.js'